In [21]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pandas.io.sql")

In [ ]:
conn = psycopg2.connect(
    dbname="retaildb",
    user="postgres",
    password="*****",
    host="localhost",
    port="5433"
)
cur = conn.cursor()

### How are sales changing month over month?

In [53]:
per_change_in_sales = pd.read_sql(""" 
WITH monthly_sales AS (
    SELECT 
        d.order_year,
        d.order_month,
        SUM(f.quantity * f.unit_price) AS total_sales
    FROM fact_sales f
    JOIN dim_order_date d ON d.date_key = f.date_key
    GROUP BY d.order_year, d.order_month
)
SELECT 
    order_year,
    order_month,
    total_sales,
    LAG(total_sales) OVER(ORDER BY order_year, order_month) AS prev_sales,
    ROUND(
        ( (total_sales - LAG(total_sales) OVER(ORDER BY order_year, order_month)) 
          / NULLIF(LAG(total_sales) OVER(ORDER BY order_year, order_month), 0) ) * 100, 
        2
    ) AS pct_change
FROM monthly_sales
ORDER BY order_year, order_month;
""", conn)
per_change_in_sales.head(10)

,order_year,order_month,total_sales,prev_sales,pct_change
0,2024,9,1500.0,NaN,NaN
1,2024,10,850.0,1500.0,-43.33
2,2024,11,12800.0,850.0,1405.88
3,2024,12,7450.0,12800.0,-41.80
4,2025,1,5550.0,7450.0,-25.50
5,2025,2,8180.0,5550.0,47.39
6,2025,3,4790.0,8180.0,-41.44
7,2025,4,6770.0,4790.0,41.34
8,2025,5,8720.0,6770.0,28.80
9,2025,6,12820.0,8720.0,47.02


### Which products have declining sales trends?

In [54]:
declining_products = pd.read_sql(""" 
WITH monthly_sales AS (
    SELECT 
        p.product_name,
        d.order_year,
        d.order_month,
        SUM(f.quantity * f.unit_price) AS total_sales
    FROM fact_sales f
    JOIN dim_products p ON p.product_key = f.product_key
    JOIN dim_order_date d ON d.date_key = f.date_key
    GROUP BY p.product_name, d.order_year, d.order_month
),
previous_sales AS (
    SELECT product_name, order_year, order_month, total_sales,
           LAG(total_sales) OVER(PARTITION BY product_name ORDER BY order_year, order_month) AS prev_sales
    FROM monthly_sales
)
SELECT product_name, order_year, order_month, total_sales, prev_sales
FROM previous_sales
WHERE total_sales < prev_sales
ORDER BY order_year DESC, order_month DESC;
""", conn)
declining_products.head(10)

,product_name,order_year,order_month,total_sales,prev_sales
0,Headphones,2025,9,400.0,500.0
1,Smartwatch,2025,9,150.0,750.0
2,Headphones,2025,8,500.0,800.0
3,Keyboard,2025,7,50.0,250.0
4,Monitor,2025,7,600.0,1800.0
5,Smartphone,2025,7,2000.0,3500.0
6,Tablet,2025,6,300.0,2700.0
7,Headphones,2025,6,800.0,900.0
8,Keyboard,2025,6,250.0,300.0
9,Printer,2025,5,120.0,720.0


### How many unique customers placed orders in a given time period?

In [55]:
unique_customers = pd.read_sql(""" 
SELECT 
    d.order_year,
    COUNT(DISTINCT f.customer_key) AS unique_customers
FROM fact_sales f
JOIN dim_order_date d ON d.date_key = f.date_key
GROUP BY d.order_year
ORDER BY d.order_year;
""", conn)
unique_customers.head()

,order_year,unique_customers
0,2024,13
1,2025,21


### Which products generate the most revenue?

In [56]:
top_products = pd.read_sql("""
SELECT 
    p.product_name, 
    d.order_year,
    SUM(f.quantity * f.unit_price) AS total_sales,
    SUM(f.quantity) AS total_quantity
FROM fact_sales f
JOIN dim_products p ON p.product_key = f.product_key
JOIN dim_order_date d ON d.date_key = f.date_key
GROUP BY d.order_year, p.product_name
ORDER BY total_sales DESC;
""", conn)
top_products.head()

,product_name,order_year,total_sales,total_quantity
0,Laptop,2025,20000.0,25
1,Camera,2025,13050.0,29
2,Smartphone,2025,12000.0,24
3,Laptop,2024,8800.0,11
4,Tablet,2025,8700.0,29


### Which shippers handle the most orders?

In [57]:
top_shippers = pd.read_sql(""" 
SELECT
    s.shipper_name,
    COUNT(*) AS total_orders
FROM fact_sales f
JOIN dim_shippers s ON s.shipper_key = f.shipper_key
GROUP BY s.shipper_name
ORDER BY total_orders DESC;
""", conn)
top_shippers.head()

,shipper_name,total_orders
0,DHL,30
1,FedEx,28
2,GLS,22
3,UPS,21


### What’s the average order size per product?

In [58]:
avg_order_value = pd.read_sql(""" 
SELECT 
    p.product_name,
    ROUND(AVG(f.quantity * f.unit_price),2) AS avg_order_value
FROM fact_sales f
JOIN dim_products p ON p.product_key = f.product_key
GROUP BY p.product_name
ORDER BY avg_order_value DESC;
""", conn)
avg_order_value.head()

,product_name,avg_order_value
0,Laptop,1920.00
1,Smartphone,1708.33
2,Camera,1687.50
3,Tablet,954.55
4,Monitor,581.82


### What is the total sales amount and quantity sold (customer,daily, monthly, yearly)?

In [59]:
sales_by_customer = pd.read_sql("""
    SELECT 
        c.customer_id, 
        c.city, 
        SUM(f.quantity * f.unit_price) AS total_sales,
        SUM(f.quantity) AS total_quantity
    FROM fact_sales f
    JOIN dim_customers c ON c.customer_key = f.customer_key
    GROUP BY c.customer_id, c.city
    ORDER BY total_quantity DESC
""", conn)
sales_by_customer.head() 


,customer_id,city,total_sales,total_quantity
0,1014,East Sarah,14100.0,34
1,1006,Jakebury,6680.0,26
2,1018,Whiteville,8950.0,20
3,1016,Owensmouth,3200.0,19
4,1013,Meghanfort,8050.0,18


In [60]:
sales_by_year = pd.read_sql("""
    SELECT 
	    d.order_year, 
	    SUM(f.quantity * f.unit_price) AS total_sales,
	    SUM(f.quantity) AS total_quantity
    FROM fact_sales f
    JOIN dim_order_date d ON d.date_key = f.date_key
    GROUP BY order_year
    ORDER BY total_quantity DESC
""", conn)
sales_by_year.head()

,order_year,total_sales,total_quantity
0,2025,71060.0,250
1,2024,22600.0,60


In [61]:
sales_by_month = pd.read_sql("""
    SELECT 
	    d.order_month, 
	    SUM(f.quantity * f.unit_price) AS total_sales,
	    SUM(f.quantity) AS total_quantity
    FROM fact_sales f
    JOIN dim_order_date d ON d.date_key = f.date_key
    GROUP BY order_month
    ORDER BY total_quantity DESC
""", conn)
sales_by_month.head()

,order_month,total_sales,total_quantity
0,2,8180.0,47
1,6,12820.0,44
2,5,8720.0,32
3,11,12800.0,29
4,9,11890.0,27


In [62]:
sales_by_day = pd.read_sql("""
    SELECT 
	    d.order_day, 
	    SUM(f.quantity * f.unit_price) AS total_sales,
	    SUM(f.quantity) AS total_quantity
    FROM fact_sales f
    JOIN dim_order_date d ON d.date_key = f.date_key
    GROUP BY d.order_day
    ORDER BY total_quantity DESC,total_sales  DESC
""", conn)
sales_by_day.head()

,order_day,total_sales,total_quantity
0,30,4750.0,23
1,6,4850.0,19
2,24,3840.0,19
3,14,6400.0,17
4,17,6050.0,17


### How many orders are completed, pending, or canceled?

In [63]:
order_status_count = pd.read_sql(""" 
SELECT
    os.order_status,
    COUNT(*) AS total_count
FROM fact_sales f
JOIN dim_order_statuses os ON os.order_status_key = f.order_status_key
GROUP BY os.order_status
ORDER BY total_count DESC;
""", conn)
order_status_count.head()

,order_status,total_count
0,Pending,26
1,Cancelled,26
2,Delivered,20
3,Processing,17
4,Shipped,12
